In [1]:
%matplotlib inline

from matplotlib import pyplot as plt
from matplotlib import patches as mpatches
import numpy as np
import pandas as pd
import math
import sys

In [2]:
data = pd.read_csv("data.csv", index_col="Ind.", sep=",")

In [69]:
P_A = 1e5 # atmosphere pressure WARNING (can be not 1e5)

def init():
    global P_A
    sys.stderr.write("P_a = %f" % (P_A))
    
    x = lambda T: T / 1e4
    
    data["sigmaN2"] = (data["sigma"] + data["sigma"]["N2"]) / 2.0
    data["epsilonN2"] = np.sqrt(data["epsilon"] * data["epsilon"]["N2"])
    data["muN2"] =  2 * data["mu"]["N2"] * data["mu"] / (data["mu"]["N2"] + data["mu"])
    
    def generate(subst):
        data.loc[subst, "omega"] = lambda T: 1.074 * np.power(T / data["epsilonN2"][subst], -0.1604)        
        data.loc[subst, "Phi"] = lambda T: \
                      data["f1"][subst] \
                    + data["f2"][subst] * np.log(x(T)) \
                    + data["f3"][subst] * np.power(x(T), -2.0) \
                    + data["f4"][subst] * np.power(x(T), -1.0) \
                    + data["f5"][subst] * x(T) \
                    + data["f6"][subst] * np.power(x(T), 2.0) \
                    + data["f7"][subst] * np.power(x(T), 3.0)
                    
        data.loc[subst, "G"] = lambda T: data["H(298)"][subst] - data["Phi"][subst](T) * T
        
        data.loc[subst, "D"] = lambda T: 2.628e-2 * np.power(T, 3.0 / 2.0) / \
                        (P_A * data["sigmaN2"][subst] * data["omega"][subst](T) * np.sqrt(data["muN2"][subst]))
            
    closure = [generate(subst) for subst in data.index]
    
init()

P_a = 100000.000000

In [10]:
# Task1
# 0 -- AlCl
# 1 -- AlCl2
# 2 -- AlCl3
# 3 -- H2
# 4 -- HCl

def solve(T):

    # 0 :::  (PHCl ** 2) - K0 * (PAlCl ** 2) * PH2 = 0
    J[0][0] = lambda x: 0 - 2 * K[0](T) * x[0] * x[3]
    J[0][1] = lambda x: 0
    J[0][2] = lambda x: 0
    J[0][3] = lambda x: 0 - K[0](T) * x[0] * x[0]
    J[0][4] = lambda x: 2 * x[4]
    
    # 1 ::: (PHCl ** 2) - K1 * PAlCl2 * PH2 = 0
    J[1][0] = lambda x: 0
    J[1][1] = lambda x: 0 - K[1](T) * x[3]
    J[1][2] = lambda x: 0
    J[1][3] = lambda x: 0 - K[1](T) * x[1]
    J[1][4] = lambda x: 2 * x[4]
    
    # 2 ::: (PHCl ** 6) - K2 * (PAlCl3 ** 2) * (PH2 ** 3)
    J[2][0] = lambda x: 0
    J[2][1] = lambda x: 0
    J[2][2] = lambda x: 0 - 2 * K[2](T) * x[2] * (x[3] ** 3)
    J[2][3] = lambda x: 0 - 3 * K[2](T) * (x[2] ** 2) * (x[3] ** 2)
    J[2][4] = lambda x: 6 * (x[4] ** 5)
    
    # 3 ::: DHCl(PgHCl - PeHCl) + 2 * DH2 * (PgH2 - PeH2) = 0
    J[3][0] = lambda x: 0
    J[3][1] = lambda x: 0
    J[3][2] = lambda x: 0
    J[3][3] = lambda x: -2 * D[3](T)
    J[3][4] = lambda x: -1 * D[4](T)
    
    # 4 ::: DAlCl(Pg(AlCl) - Pe(AlCl)) 
    #   + 2 * D(AlCl2)(Pg(AlCl2) - Pe(AlCl2)) 
    #   + 3 * D(AlCl3) * (Pg(AlCl3) - Pe(AlCl3)) 
    #   + D(HCl) * (Pg(HCl) - Pe(HCl)) = 0
    J[3][0] = lambda x: -D[0](T)
    J[3][1] = lambda x: -2 * D[1](T)
    J[3][2] = lambda x: -3 * D[2](T)
    J[3][3] = lambda x: 0
    J[3][4] = lambda x: -D[4](T)


In [39]:
data["muN2"]

Ind.
AlCl     38.674351
AlCl2    43.560758
AlCl3    46.299864
GaCl     44.242680
GaCl2    46.720092
GaCl3    48.336837
NH3      21.183404
H2        3.761316
HCl      31.683851
N2       28.013500
Al       27.487817
Ga       39.968390
AlN      33.280938
GaN      41.981330
Name: muN2, dtype: float64

In [77]:
data["G"]["NH3"](365)

-116513.70198428207

1.0158422196040229

38.67435135658057